# 🔍 Complex RAG Pipeline with LangChain, Multiple Tools, and LLMs

In [11]:
!pip install langchain openai faiss-cpu

In [12]:
!pip install tiktoken
!pip install langchain_community

In [13]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

In [14]:

# Set your API keys here
os.environ["OPENAI_API_KEY"] = ""


In [15]:

# === Sample Document ===
doc = Document(page_content="""
Generative AI models like GPT-4 and Claude are used to generate natural language text.
RAG systems enhance LLMs by retrieving relevant documents from a knowledge base.
FAISS is a popular open-source vector database for similarity search.
LangChain enables modular and composable LLM workflows.
""")


In [16]:

# === Split and Embed ===
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
docs = splitter.split_documents([doc])

embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding_model)


In [17]:
# === Tool 1: Dummy Web Search Tool ===
def dummy_search_tool(query: str) -> str:
    return f"Simulated search result for: {query}"

search_tool = Tool(
    name="WebSearch",
    func=dummy_search_tool,
    description="Search the web when user asks about current events."
)

In [18]:

# === Tool 2: Vector Store QA Tool ===
retriever = vectorstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0),
    retriever=retriever
)

rag_tool = Tool(
    name="VectorDB QA",
    func=qa_chain.run,
    description="Answer questions from internal knowledge base."
)


In [19]:

# === Prompt + Agent ===
llm_smart = ChatOpenAI(model_name="gpt-4", temperature=0.3)
prompt = PromptTemplate(input_variables=["input"], template="Answer: {input}")
llm_chain = LLMChain(llm=llm_smart, prompt=prompt)

tools = [search_tool, rag_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm_smart,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


In [20]:

# === Run the Agent ===
response = agent.run("What is LangChain and how does it relate to vector databases?")
print("📌 Agent Response:", response)




> Entering new AgentExecutor chain...
This question seems to be asking about a specific technology or concept, which might be found in our internal knowledge base.
Action: VectorDB QA
Action Input: What is LangChain and how does it relate to vector databases?
Observation: LangChain is a system that enables modular and composable workflows for large language models (LLMs). It helps in structuring and organizing the workflow of LLMs for various tasks. LangChain itself does not directly relate to vector databases like FAISS. Vector databases like FAISS are used for similarity search and retrieval of vectors, which can be useful in tasks like information retrieval or recommendation systems. While LangChain can potentially be used in conjunction with vector databases in certain applications, they serve different purposes in the context of natural language processing workflows.
Thought:The information from the internal knowledge base provides a clear explanation of LangChain and its relati